***GENERATED CODE FOR thyroiddosage PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('Patientid')
        transformationDF = transformationDF.drop('Visit Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'Gender'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '168', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'transformation_label': 'String Indexer', 'feature_label': 'Gender'})
        transformationDF = transformationDF.drop('Gender')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run thyroiddosageHooks.ipynb
try:
	#sourcePreExecutionHook()

	thyroiddataset = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'fileType': 'Delimeted', 'domain': 'http://35.230.65.179', 'baseType': 'hdfs', 'url': '/FileStore/platform/uploadedSourceFiles/thyroid_dataset.csv', 'filename': 'thyroid_dataset.csv', 'token': '1', 'delimiter': ','}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run thyroiddosageHooks.ipynb
try:
	#transformationPreExecutionHook()

	manualfe = TransformationMain.run(thyroiddataset,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Patientid", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "168", "mean": "14790.29", "stddev": "6994.25", "min": "144", "max": "27482", "missing": "0"}, "updatedLabel": "Patientid"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Visit Date", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "168", "mean": "", "stddev": "", "min": "10/1/2019", "max": "9/9/2016", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Visit Date"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "Gender"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "168", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Dosage", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "168", "mean": "84.96", "stddev": "52.23", "min": "5.0", "max": "400.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Dosage"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Gender_Num", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "168", "mean": "0.27", "stddev": "0.44", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_Num"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Height", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "168", "mean": "154.07", "stddev": "8.04", "min": "120.5", "max": "178.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Height"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Weight", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "168", "mean": "65.59", "stddev": "10.89", "min": "37.3", "max": "96.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Weight"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "168", "mean": "36.0", "stddev": "11.67", "min": "12", "max": "70", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BMI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "168", "mean": "27.48", "stddev": "4.59", "min": "16.22", "max": "39.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BMI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Baseline TSH", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "168", "mean": "8.01", "stddev": "6.98", "min": "0.005", "max": "26.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Baseline TSH"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Baseline T3", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "168", "mean": "1.34", "stddev": "0.79", "min": "0.3", "max": "3.677763851", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Baseline T3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Baseline T4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "168", "mean": "8.2", "stddev": "2.52", "min": "1.4", "max": "14.75", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Baseline T4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Baseline FT4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "168", "mean": "1.19", "stddev": "0.31", "min": "0.33", "max": "1.96", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Baseline FT4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TSH After Dosage", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "168", "mean": "5.74", "stddev": "5.5", "min": "0.005", "max": "20.98", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TSH After Dosage"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "T3 After Dosage", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "168", "mean": "1.22", "stddev": "0.44", "min": "0.62", "max": "2.5826563730000003", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "T3 After Dosage"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "T4 After Dosage", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "168", "mean": "8.92", "stddev": "2.37", "min": "1.74", "max": "15.82", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "T4 After Dosage"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FT4 After Dosage", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "168", "mean": "1.27", "stddev": "0.29", "min": "0.51", "max": "2.03", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "FT4 After Dosage"}]}))

	#transformationPostExecutionHook(manualfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run thyroiddosageHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(manualfe, ["Gender_stringindexer", "Gender_Num", "Height", "Weight", "Age", "BMI", "Baseline TSH", "Baseline T3", "Baseline T4", "Baseline FT4", "TSH After Dosage", "T3 After Dosage", "T4 After Dosage", "FT4 After Dosage"], "Dosage")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

